#### Problem Statement:- The Toxic Pesticides
Though, many of us don't appreciate much, but a farmer's job is real test of endurance and determination. Once the seeds are sown, he works days and nights to make sure that he cultivates a good harvest at the end of season. A good harvest is ensured by several factors such as availability of water, soil fertility, protecting crops from rodents, timely use of pesticides & other useful chemicals and nature. While a lot of these factors are difficult to control for, the amount and frequency of pesticides is something the farmer can control.

Pesticides are also special, because while they protect the crop with the right dosage. But, if you add more than required, they may spoil the entire harvest. A high level of pesticide can deem the crop dead / unsuitable for consumption among many outcomes. This data is based on crops harvested by various farmers at the end of harvest season. To simplify the problem, you can assume that all other factors like variations in farming techniques have been controlled for.

You need to daetermine the outcome of the harvest season, i.e. whether the crop would be healthy (alive), damaged by pesticides or damaged by other reasons.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
def model_evaluation(actuals, predictions) :
    
    print('\nConfusion Matrix :')
    print(confusion_matrix(actuals, predictions) ) 
    
    print('\nReport : ')
    print(classification_report(actuals, predictions))
    
    print('\nAccuracy Score :', accuracy_score(actuals, predictions)) 
    print("------------------------------------------------------------------")

In [20]:
df_train=pd.read_csv("/content/drive/My Drive/aggr/train.csv", sep=",",na_values=["","#","?","unknown","None"])
df_test=pd.read_csv("/content/drive/My Drive/aggr/test.csv", sep=",",na_values=["","#","?","unknown","None"])

In [21]:
df_train['train_flag'] = 1
df_test['train_flag'] = 0
df_test['Crop_Damage'] = 0
print(df_train.shape, df_test.shape)

df_data = pd.concat((df_train, df_test))
print(df_data.shape)

(88858, 11) (59310, 11)
(148168, 11)


In [22]:
feature_cols = df_train.columns.tolist()
feature_cols.remove('ID')
feature_cols.remove('Crop_Damage')
feature_cols.remove('train_flag')
label_col = 'Crop_Damage'
print(feature_cols)

['Estimated_Insects_Count', 'Crop_Type', 'Soil_Type', 'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used', 'Number_Weeks_Quit', 'Season']


In [23]:
df_data['ID_value'] = df_data['ID'].apply(lambda x: x.strip('F')).astype('int')

In [24]:
df_data = df_data.sort_values(['ID_value'])
df_data = df_data.reset_index(drop=True)

In [25]:
df_data['Soil_Type_Damage'] = df_data.sort_values(['ID_value']).groupby(['Soil_Type'])['Crop_Damage'].apply(lambda x: x.shift().rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Estimated_Insects_Count_Damage'] = df_data.sort_values(['ID_value']).groupby(['Estimated_Insects_Count'])['Crop_Damage'].apply(lambda x: x.shift().rolling(5, min_periods=1).mean()).fillna(-999).values


df_data['Crop_Type_Damage'] = df_data.sort_values(['ID_value']).groupby(['Crop_Type'])['Crop_Damage'].apply(lambda x: x.shift().rolling(5, min_periods=1).mean()).fillna(-999).values


df_data['Pesticide_Use_Category_Damage'] = df_data.sort_values(['ID_value']).groupby(['Pesticide_Use_Category'])['Crop_Damage'].apply(lambda x: x.shift().rolling(5, min_periods=1).mean()).fillna(-999).values


df_data['Season_Damage'] = df_data.sort_values(['ID_value']).groupby(['Season'])['Crop_Damage'].apply(lambda x: x.shift().rolling(5, min_periods=1).mean()).fillna(-999).values


df_data['Soil_Type_Damage_lag2'] = df_data.sort_values(['ID_value']).groupby(['Soil_Type'])['Crop_Damage'].apply(lambda x: x.shift(periods=2).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Estimated_Insects_Count_Damage_lag2'] = df_data.sort_values(['ID_value']).groupby(['Estimated_Insects_Count'])['Crop_Damage'].apply(lambda x: x.shift(periods=2).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Crop_Type_Damage_lag2'] = df_data.sort_values(['ID_value']).groupby(['Crop_Type'])['Crop_Damage'].apply(lambda x: x.shift(periods=2).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Pesticide_Use_Category_Damage_lag2'] = df_data.sort_values(['ID_value']).groupby(['Pesticide_Use_Category'])['Crop_Damage'].apply(lambda x: x.shift(periods=2).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Season_Damage_lag2'] = df_data.sort_values(['ID_value']).groupby(['Season'])['Crop_Damage'].apply(lambda x: x.shift(periods=2).rolling(5, min_periods=1).mean()).fillna(-999).values

In [26]:
df_data['Soil_Type_Damage_down'] = df_data.sort_values(['ID_value']).groupby(['Soil_Type'])['Crop_Damage'].apply(lambda x: x.shift(-1).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Estimated_Insects_Count_Damage_down'] = df_data.sort_values(['ID_value']).groupby(['Estimated_Insects_Count'])['Crop_Damage'].apply(lambda x: x.shift(-1).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Crop_Type_Damage_down'] = df_data.sort_values(['ID_value']).groupby(['Crop_Type'])['Crop_Damage'].apply(lambda x: x.shift(-1).rolling(5, min_periods=1).mean()).fillna(-999).values


df_data['Pesticide_Use_Category_Damage_down'] = df_data.sort_values(['ID_value']).groupby(['Pesticide_Use_Category'])['Crop_Damage'].apply(lambda x: x.shift(-1).rolling(5, min_periods=1).mean()).fillna(-999).values


df_data['Season_Damage_down'] = df_data.sort_values(['ID_value']).groupby(['Season'])['Crop_Damage'].apply(lambda x: x.shift(-1).rolling(5, min_periods=1).mean()).fillna(-999).values


df_data['Soil_Type_Damage_lag2_down'] = df_data.sort_values(['ID_value']).groupby(['Soil_Type'])['Crop_Damage'].apply(lambda x: x.shift(periods=-2).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Estimated_Insects_Count_Damage_lag2_down'] = df_data.sort_values(['ID_value']).groupby(['Estimated_Insects_Count'])['Crop_Damage'].apply(lambda x: x.shift(periods=-2).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Crop_Type_Damage_lag2_down'] = df_data.sort_values(['ID_value']).groupby(['Crop_Type'])['Crop_Damage'].apply(lambda x: x.shift(periods=-2).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Pesticide_Use_Category_Damage_lag2_down'] = df_data.sort_values(['ID_value']).groupby(['Pesticide_Use_Category'])['Crop_Damage'].apply(lambda x: x.shift(periods=-2).rolling(5, min_periods=1).mean()).fillna(-999).values

df_data['Season_Damage_lag2_down'] = df_data.sort_values(['ID_value']).groupby(['Season'])['Crop_Damage'].apply(lambda x: x.shift(periods=-2).rolling(5, min_periods=1).mean()).fillna(-999).values

In [27]:
df_data.loc[df_data['train_flag'] == 0, 'Crop_Damage'] = -999

In [28]:
df_data.isnull().sum()

ID                                              0
Estimated_Insects_Count                         0
Crop_Type                                       0
Soil_Type                                       0
Pesticide_Use_Category                          0
Number_Doses_Week                               0
Number_Weeks_Used                           14893
Number_Weeks_Quit                               0
Season                                          0
Crop_Damage                                     0
train_flag                                      0
ID_value                                        0
Soil_Type_Damage                                0
Estimated_Insects_Count_Damage                  0
Crop_Type_Damage                                0
Pesticide_Use_Category_Damage                   0
Season_Damage                                   0
Soil_Type_Damage_lag2                           0
Estimated_Insects_Count_Damage_lag2             0
Crop_Type_Damage_lag2                           0


In [29]:
df_data['Crop_Damage_lag1'] = df_data['Crop_Damage'].shift(fill_value=-999)
df_data['Estimated_Insects_Count_lag1'] = df_data['Estimated_Insects_Count'].shift(fill_value=-999)
df_data['Crop_Type_lag1'] = df_data['Crop_Type'].shift(fill_value=-999)
df_data['Soil_Type_lag1'] = df_data['Soil_Type'].shift(fill_value=-999)
df_data['Pesticide_Use_Category_lag1'] = df_data['Pesticide_Use_Category'].shift(fill_value=-999)
df_data['Number_Doses_Week_lag1'] = df_data['Number_Doses_Week'].shift(fill_value=-999)
df_data['Number_Weeks_Used_lag1'] = df_data['Number_Weeks_Used'].shift(fill_value=-999)
df_data['Number_Weeks_Quit_lag1'] = df_data['Number_Weeks_Quit'].shift(fill_value=-999)
df_data['Season_lag1'] = df_data['Season'].shift(fill_value=-999)

df_data['Crop_Damage_lag2'] = df_data['Crop_Damage'].shift(periods=2,fill_value=-999)
df_data['Estimated_Insects_Count_lag2'] = df_data['Estimated_Insects_Count'].shift(periods=2,fill_value=-999)
df_data['Crop_Type_lag2'] = df_data['Crop_Type'].shift(fill_value=-999)
df_data['Soil_Type_lag2'] = df_data['Soil_Type'].shift(fill_value=-999)
df_data['Pesticide_Use_Category_lag2'] = df_data['Pesticide_Use_Category'].shift(periods=2,fill_value=-999)
df_data['Number_Doses_Week_lag2'] = df_data['Number_Doses_Week'].shift(periods=2,fill_value=-999)
df_data['Number_Weeks_Used_lag2'] = df_data['Number_Weeks_Used'].shift(periods=2,fill_value=-999)
df_data['Number_Weeks_Quit_lag2'] = df_data['Number_Weeks_Quit'].shift(periods=2,fill_value=-999)
df_data['Season_lag2'] = df_data['Season'].shift(periods=2,fill_value=-999)


In [30]:
df_data.isnull().sum()

ID                                              0
Estimated_Insects_Count                         0
Crop_Type                                       0
Soil_Type                                       0
Pesticide_Use_Category                          0
Number_Doses_Week                               0
Number_Weeks_Used                           14893
Number_Weeks_Quit                               0
Season                                          0
Crop_Damage                                     0
train_flag                                      0
ID_value                                        0
Soil_Type_Damage                                0
Estimated_Insects_Count_Damage                  0
Crop_Type_Damage                                0
Pesticide_Use_Category_Damage                   0
Season_Damage                                   0
Soil_Type_Damage_lag2                           0
Estimated_Insects_Count_Damage_lag2             0
Crop_Type_Damage_lag2                           0


In [31]:
df_train, df_test = df_data[df_data.train_flag == 1], df_data[df_data.train_flag == 0]

In [32]:
df_train.drop(['train_flag'], inplace=True, axis=1)
df_test.drop(['train_flag'], inplace=True, axis=1)
df_test.drop([label_col], inplace=True, axis=1)

In [33]:
print(df_train.shape, df_test.shape)

(88858, 49) (59310, 48)


In [34]:
missing_impute = -999

In [35]:
df_test.isnull().sum()

ID                                             0
Estimated_Insects_Count                        0
Crop_Type                                      0
Soil_Type                                      0
Pesticide_Use_Category                         0
Number_Doses_Week                              0
Number_Weeks_Used                           5893
Number_Weeks_Quit                              0
Season                                         0
ID_value                                       0
Soil_Type_Damage                               0
Estimated_Insects_Count_Damage                 0
Crop_Type_Damage                               0
Pesticide_Use_Category_Damage                  0
Season_Damage                                  0
Soil_Type_Damage_lag2                          0
Estimated_Insects_Count_Damage_lag2            0
Crop_Type_Damage_lag2                          0
Pesticide_Use_Category_Damage_lag2             0
Season_Damage_lag2                             0
Soil_Type_Damage_dow

In [36]:
df_train['Number_Weeks_Used'] = df_train['Number_Weeks_Used'].apply(lambda x: missing_impute if pd.isna(x) else x)
df_test['Number_Weeks_Used'] = df_test['Number_Weeks_Used'].apply(lambda x: missing_impute if pd.isna(x) else x)

df_train['Number_Weeks_Used_lag1'] = df_train['Number_Weeks_Used_lag1'].apply(lambda x: missing_impute if pd.isna(x) else x)
df_test['Number_Weeks_Used_lag1'] = df_test['Number_Weeks_Used_lag1'].apply(lambda x: missing_impute if pd.isna(x) else x)

df_train['Number_Weeks_Used_lag2'] = df_train['Number_Weeks_Used_lag2'].apply(lambda x: missing_impute if pd.isna(x) else x)
df_test['Number_Weeks_Used_lag2'] = df_test['Number_Weeks_Used_lag2'].apply(lambda x: missing_impute if pd.isna(x) else x)

In [37]:
df_train, df_eval = train_test_split(df_train, test_size=0.40, random_state=42, shuffle=True, stratify=df_train[label_col])

In [38]:
feature_cols = df_train.columns.tolist()
feature_cols.remove('ID')
feature_cols.remove('Crop_Damage')
feature_cols.remove('ID_value')
label_col = 'Crop_Damage'
print(feature_cols)

['Estimated_Insects_Count', 'Crop_Type', 'Soil_Type', 'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used', 'Number_Weeks_Quit', 'Season', 'Soil_Type_Damage', 'Estimated_Insects_Count_Damage', 'Crop_Type_Damage', 'Pesticide_Use_Category_Damage', 'Season_Damage', 'Soil_Type_Damage_lag2', 'Estimated_Insects_Count_Damage_lag2', 'Crop_Type_Damage_lag2', 'Pesticide_Use_Category_Damage_lag2', 'Season_Damage_lag2', 'Soil_Type_Damage_down', 'Estimated_Insects_Count_Damage_down', 'Crop_Type_Damage_down', 'Pesticide_Use_Category_Damage_down', 'Season_Damage_down', 'Soil_Type_Damage_lag2_down', 'Estimated_Insects_Count_Damage_lag2_down', 'Crop_Type_Damage_lag2_down', 'Pesticide_Use_Category_Damage_lag2_down', 'Season_Damage_lag2_down', 'Crop_Damage_lag1', 'Estimated_Insects_Count_lag1', 'Crop_Type_lag1', 'Soil_Type_lag1', 'Pesticide_Use_Category_lag1', 'Number_Doses_Week_lag1', 'Number_Weeks_Used_lag1', 'Number_Weeks_Quit_lag1', 'Season_lag1', 'Crop_Damage_lag2', 'Estimated_Insects_

In [39]:
cat_cols = ['Crop_Type', 'Soil_Type', 'Pesticide_Use_Category', 'Season', 'Crop_Type_lag1', 'Soil_Type_lag1', 'Pesticide_Use_Category_lag1', 'Season_lag1']

In [40]:
num=list(df_train.columns.difference(cat_cols))
num

['Crop_Damage',
 'Crop_Damage_lag1',
 'Crop_Damage_lag2',
 'Crop_Type_Damage',
 'Crop_Type_Damage_down',
 'Crop_Type_Damage_lag2',
 'Crop_Type_Damage_lag2_down',
 'Crop_Type_lag2',
 'Estimated_Insects_Count',
 'Estimated_Insects_Count_Damage',
 'Estimated_Insects_Count_Damage_down',
 'Estimated_Insects_Count_Damage_lag2',
 'Estimated_Insects_Count_Damage_lag2_down',
 'Estimated_Insects_Count_lag1',
 'Estimated_Insects_Count_lag2',
 'ID',
 'ID_value',
 'Number_Doses_Week',
 'Number_Doses_Week_lag1',
 'Number_Doses_Week_lag2',
 'Number_Weeks_Quit',
 'Number_Weeks_Quit_lag1',
 'Number_Weeks_Quit_lag2',
 'Number_Weeks_Used',
 'Number_Weeks_Used_lag1',
 'Number_Weeks_Used_lag2',
 'Pesticide_Use_Category_Damage',
 'Pesticide_Use_Category_Damage_down',
 'Pesticide_Use_Category_Damage_lag2',
 'Pesticide_Use_Category_Damage_lag2_down',
 'Pesticide_Use_Category_lag2',
 'Season_Damage',
 'Season_Damage_down',
 'Season_Damage_lag2',
 'Season_Damage_lag2_down',
 'Season_lag2',
 'Soil_Type_Damage',


In [41]:
remove_list=['Crop_Damage','ID',]

In [42]:
num = [i for i in num if i not in remove_list]
num

['Crop_Damage_lag1',
 'Crop_Damage_lag2',
 'Crop_Type_Damage',
 'Crop_Type_Damage_down',
 'Crop_Type_Damage_lag2',
 'Crop_Type_Damage_lag2_down',
 'Crop_Type_lag2',
 'Estimated_Insects_Count',
 'Estimated_Insects_Count_Damage',
 'Estimated_Insects_Count_Damage_down',
 'Estimated_Insects_Count_Damage_lag2',
 'Estimated_Insects_Count_Damage_lag2_down',
 'Estimated_Insects_Count_lag1',
 'Estimated_Insects_Count_lag2',
 'ID_value',
 'Number_Doses_Week',
 'Number_Doses_Week_lag1',
 'Number_Doses_Week_lag2',
 'Number_Weeks_Quit',
 'Number_Weeks_Quit_lag1',
 'Number_Weeks_Quit_lag2',
 'Number_Weeks_Used',
 'Number_Weeks_Used_lag1',
 'Number_Weeks_Used_lag2',
 'Pesticide_Use_Category_Damage',
 'Pesticide_Use_Category_Damage_down',
 'Pesticide_Use_Category_Damage_lag2',
 'Pesticide_Use_Category_Damage_lag2_down',
 'Pesticide_Use_Category_lag2',
 'Season_Damage',
 'Season_Damage_down',
 'Season_Damage_lag2',
 'Season_Damage_lag2_down',
 'Season_lag2',
 'Soil_Type_Damage',
 'Soil_Type_Damage_down

In [43]:
df_train.dtypes

ID                                           object
Estimated_Insects_Count                       int64
Crop_Type                                     int64
Soil_Type                                     int64
Pesticide_Use_Category                        int64
Number_Doses_Week                             int64
Number_Weeks_Used                           float64
Number_Weeks_Quit                             int64
Season                                        int64
Crop_Damage                                   int64
ID_value                                      int64
Soil_Type_Damage                            float64
Estimated_Insects_Count_Damage              float64
Crop_Type_Damage                            float64
Pesticide_Use_Category_Damage               float64
Season_Damage                               float64
Soil_Type_Damage_lag2                       float64
Estimated_Insects_Count_Damage_lag2         float64
Crop_Type_Damage_lag2                       float64
Pesticide_Us

In [44]:
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(initial_strategy='median')),
    ('scaler', StandardScaler()) ])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent',fill_value="missing_value")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num),
        ('cat', categorical_transformer, cat_cols)])

In [45]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold,GridSearchCV, RandomizedSearchCV

clf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', tree.DecisionTreeClassifier(random_state=123))])
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=143)

param_grid = {"classifier__criterion": ['entropy','gini'],
              "classifier__splitter":["best", "random"],
             'classifier__min_samples_split': [5,10,20],
             'classifier__max_depth': [2,15,30,],
             'classifier__max_leaf_nodes': [50,100,120], }

rf_grid = RandomizedSearchCV(clf, param_distributions= param_grid, cv=kfold, n_jobs=-1)

rf_grid.fit(df_train,df_train[label_col])

train_predictions=rf_grid.predict(df_train)
predictions = rf_grid.predict(df_eval)

In [46]:
print(model_evaluation(df_train[label_col], train_predictions))
print(model_evaluation(df_eval[label_col], predictions))


Confusion Matrix :
[[44140   397     5]
 [  374  6859   151]
 [   26   922   440]]

Report : 
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     44542
           1       0.84      0.93      0.88      7384
           2       0.74      0.32      0.44      1388

    accuracy                           0.96     53314
   macro avg       0.86      0.75      0.77     53314
weighted avg       0.96      0.96      0.96     53314


Accuracy Score : 0.9648310012379487
------------------------------------------------------------------
None

Confusion Matrix :
[[29388   299     9]
 [  257  4509   157]
 [   18   627   280]]

Report : 
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     29696
           1       0.83      0.92      0.87      4923
           2       0.63      0.30      0.41       925

    accuracy                           0.96     35544
   macro avg       0.82      0.74      0.76     

In [ ]:
test_predictions=rf_grid.predict(df_test)


In [47]:
accuracy_score(df_train[label_col], train_predictions)

0.9648310012379487

In [48]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', XGBClassifier(n_jobs=-1,random_state=123))])
clf.fit(df_train,df_train[label_col])

train_predictions=clf.predict(df_train)
predictions = clf.predict(df_eval)

In [49]:
print(model_evaluation(df_train[label_col], train_predictions))
print(model_evaluation(df_eval[label_col], predictions))


Confusion Matrix :
[[44127   403    12]
 [  169  7153    62]
 [    0  1008   380]]

Report : 
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     44542
           1       0.84      0.97      0.90      7384
           2       0.84      0.27      0.41      1388

    accuracy                           0.97     53314
   macro avg       0.89      0.74      0.77     53314
weighted avg       0.97      0.97      0.96     53314


Accuracy Score : 0.9689762538920359
------------------------------------------------------------------
None

Confusion Matrix :
[[29384   301    11]
 [  114  4746    63]
 [    0   666   259]]

Report : 
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     29696
           1       0.83      0.96      0.89      4923
           2       0.78      0.28      0.41       925

    accuracy                           0.97     35544
   macro avg       0.87      0.74      0.77     

In [ ]:
df_test["Crop_Damage"]=0

In [66]:
df_train.columns

Index(['ID', 'Estimated_Insects_Count', 'Crop_Type', 'Soil_Type',
       'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used',
       'Number_Weeks_Quit', 'Season', 'Crop_Damage', 'ID_value',
       'Soil_Type_Damage', 'Estimated_Insects_Count_Damage',
       'Crop_Type_Damage', 'Pesticide_Use_Category_Damage', 'Season_Damage',
       'Soil_Type_Damage_lag2', 'Estimated_Insects_Count_Damage_lag2',
       'Crop_Type_Damage_lag2', 'Pesticide_Use_Category_Damage_lag2',
       'Season_Damage_lag2', 'Soil_Type_Damage_down',
       'Estimated_Insects_Count_Damage_down', 'Crop_Type_Damage_down',
       'Pesticide_Use_Category_Damage_down', 'Season_Damage_down',
       'Soil_Type_Damage_lag2_down',
       'Estimated_Insects_Count_Damage_lag2_down',
       'Crop_Type_Damage_lag2_down', 'Pesticide_Use_Category_Damage_lag2_down',
       'Season_Damage_lag2_down', 'Crop_Damage_lag1',
       'Estimated_Insects_Count_lag1', 'Crop_Type_lag1', 'Soil_Type_lag1',
       'Pesticide_Use_Cate

In [67]:
df_test=df_test[['ID', 'Estimated_Insects_Count', 'Crop_Type', 'Soil_Type',
       'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used',
       'Number_Weeks_Quit', 'Season', 'Crop_Damage', 'ID_value',
       'Soil_Type_Damage', 'Estimated_Insects_Count_Damage',
       'Crop_Type_Damage', 'Pesticide_Use_Category_Damage', 'Season_Damage',
       'Soil_Type_Damage_lag2', 'Estimated_Insects_Count_Damage_lag2',
       'Crop_Type_Damage_lag2', 'Pesticide_Use_Category_Damage_lag2',
       'Season_Damage_lag2', 'Soil_Type_Damage_down',
       'Estimated_Insects_Count_Damage_down', 'Crop_Type_Damage_down',
       'Pesticide_Use_Category_Damage_down', 'Season_Damage_down',
       'Soil_Type_Damage_lag2_down',
       'Estimated_Insects_Count_Damage_lag2_down',
       'Crop_Type_Damage_lag2_down', 'Pesticide_Use_Category_Damage_lag2_down',
       'Season_Damage_lag2_down', 'Crop_Damage_lag1',
       'Estimated_Insects_Count_lag1', 'Crop_Type_lag1', 'Soil_Type_lag1',
       'Pesticide_Use_Category_lag1', 'Number_Doses_Week_lag1',
       'Number_Weeks_Used_lag1', 'Number_Weeks_Quit_lag1', 'Season_lag1',
       'Crop_Damage_lag2', 'Estimated_Insects_Count_lag2', 'Crop_Type_lag2',
       'Soil_Type_lag2', 'Pesticide_Use_Category_lag2',
       'Number_Doses_Week_lag2', 'Number_Weeks_Used_lag2',
       'Number_Weeks_Quit_lag2', 'Season_lag2']]

In [70]:
preds=clf.predict(df_test)

In [78]:
xgb=pd.DataFrame(data=df_test["ID"],columns=["ID"])

In [79]:
xgb["Crop_Damage"]=preds

In [81]:
xgb.to_csv("xgbpreds.csv",index=False)

In [82]:
rfgrid=pd.DataFrame(data=df_test["ID"],columns=["ID"])
rfgrid["Crop_Damage"]=rf_grid.predict(df_test)


In [84]:
rfgrid.to_csv("rfgridpreds.csv",index=False)